# Training - Complete End-to-End ML Training Infrastructure

Welcome to the Training module! You'll build the complete training infrastructure that orchestrates data loading, forward passes, loss computation, backpropagation, and optimization into a unified system.

## Learning Goals
- Systems understanding: How training loops coordinate all ML system components and why training orchestration determines system reliability
- Core implementation skill: Build loss functions, evaluation metrics, and complete training loops with checkpointing and monitoring
- Pattern recognition: Understand how different loss functions affect learning dynamics and model behavior
- Framework connection: See how your training loop mirrors PyTorch's training patterns and state management
- Performance insight: Learn why training loop design affects convergence speed, memory usage, and debugging capability

## Build → Use → Reflect
1. **Build**: Complete training infrastructure with loss functions, metrics, checkpointing, and progress monitoring
2. **Use**: Train real neural networks on CIFAR-10 and achieve meaningful accuracy on complex visual tasks
3. **Reflect**: Why does training loop design often determine the success or failure of ML projects?

## What You'll Achieve
By the end of this module, you'll understand:
- Deep technical understanding of how training loops orchestrate complex ML systems into reliable, monitorable processes
- Practical capability to build production-ready training infrastructure with proper error handling and state management
- Systems insight into why training stability and reproducibility are critical for reliable ML systems
- Performance consideration of how training loop efficiency affects iteration speed and resource utilization
- Connection to production ML systems and how modern MLOps platforms build on these training patterns

## Systems Reality Check
💡 **Production Context**: Modern ML training platforms like PyTorch Lightning and Hugging Face Transformers build sophisticated abstractions on top of basic training loops to handle distributed training, mixed precision, and fault tolerance
⚡ **Performance Note**: Training loop efficiency often matters more than model efficiency for development speed - good training infrastructure accelerates the entire ML development cycle

In [ ]:
#| default_exp core.training

#| export
import numpy as np
import sys
import os
from collections import defaultdict
import time
import pickle

# Add module directories to Python path
sys.path.append(os.path.abspath('modules/source/02_tensor'))
sys.path.append(os.path.abspath('modules/source/03_activations'))
sys.path.append(os.path.abspath('modules/source/04_layers'))
sys.path.append(os.path.abspath('modules/source/05_dense'))
sys.path.append(os.path.abspath('modules/source/06_spatial'))
sys.path.append(os.path.abspath('modules/source/08_dataloader'))
sys.path.append(os.path.abspath('modules/source/09_autograd'))
sys.path.append(os.path.abspath('modules/source/10_optimizers'))

# Helper function to set up import paths
# No longer needed, will use direct relative imports

# Set up paths
# No longer needed

# Import all the building blocks we need
from tensor_dev import Tensor
from activations_dev import ReLU, Sigmoid, Tanh, Softmax
from layers_dev import Dense
from dense_dev import Sequential, create_mlp
from spatial_dev import Conv2D, flatten
from dataloader_dev import Dataset, DataLoader
from autograd_dev import Variable
from optimizers_dev import SGD, Adam, StepLR

## 🔧 DEVELOPMENT

## Step 1: Understanding Loss Functions

### What are Loss Functions?
Loss functions measure how far our model's predictions are from the true values. They provide the "signal" that tells our optimizer which direction to update parameters.

### The Mathematical Foundation
Training a neural network is an optimization problem:
```
θ* = argmin_θ L(f(x; θ), y)
```
Where:
- `θ` = model parameters (weights and biases)
- `f(x; θ)` = model predictions
- `y` = true labels
- `L` = loss function
- `θ*` = optimal parameters

### Why Loss Functions Matter
- **Optimization target**: They define what "good" means for our model
- **Gradient source**: Provide gradients for backpropagation
- **Task-specific**: Different losses for different problems
- **Training dynamics**: Shape how the model learns

### Common Loss Functions

#### **Mean Squared Error (MSE)** - For Regression
```
MSE = (1/n) * Σ(y_pred - y_true)²
```
- **Use case**: Regression problems
- **Properties**: Penalizes large errors heavily
- **Gradient**: 2 * (y_pred - y_true)

#### **Cross-Entropy Loss** - For Classification
```
CrossEntropy = -Σ y_true * log(y_pred)
```
- **Use case**: Multi-class classification
- **Properties**: Penalizes confident wrong predictions
- **Gradient**: y_pred - y_true (with softmax)

#### **Binary Cross-Entropy** - For Binary Classification
```
BCE = -y_true * log(y_pred) - (1-y_true) * log(1-y_pred)
```
- **Use case**: Binary classification
- **Properties**: Symmetric around 0.5
- **Gradient**: (y_pred - y_true) / (y_pred * (1-y_pred))

Let's implement these essential loss functions!

In [ ]:
#| export
class MeanSquaredError:
    """
    Mean Squared Error Loss for Regression
    
    Measures the average squared difference between predictions and targets.
    MSE = (1/n) * Σ(y_pred - y_true)²
    """
    
    def __init__(self):
        """Initialize MSE loss function."""
        pass
    
    def __call__(self, y_pred: Tensor, y_true: Tensor) -> Tensor:
        """
        Compute MSE loss between predictions and targets.
        
        Args:
            y_pred: Model predictions (shape: [batch_size, ...])
            y_true: True targets (shape: [batch_size, ...])
            
        Returns:
            Scalar loss value
            
        TODO: Implement Mean SquaredError loss computation.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Compute difference: diff = y_pred - y_true
        2. Square the differences: squared_diff = diff²
        3. Take mean over all elements: mean(squared_diff)
        4. Return as scalar Tensor
        
        EXAMPLE:
        y_pred = Tensor([[1.0, 2.0], [3.0, 4.0]])
        y_true = Tensor([[1.5, 2.5], [2.5, 3.5]])
        loss = mse_loss(y_pred, y_true)
        # Should return: mean([(1.0-1.5)², (2.0-2.5)², (3.0-2.5)², (4.0-3.5)²])
        #                = mean([0.25, 0.25, 0.25, 0.25]) = 0.25
        
        LEARNING CONNECTIONS:
        - **Regression Optimization**: MSE loss guides models toward accurate numerical predictions
        - **Gradient Properties**: MSE provides smooth gradients proportional to prediction error
        - **Outlier Sensitivity**: Squared errors heavily penalize large mistakes
        - **Production Usage**: Common in recommendation systems, time series, and financial modeling
        
        HINTS:
        - Use tensor subtraction: y_pred - y_true
        - Use tensor power: diff ** 2
        - Use tensor mean: squared_diff.mean()
        """
        ### BEGIN SOLUTION
        diff = y_pred - y_true
        squared_diff = diff * diff  # Using multiplication for square
        loss = np.mean(squared_diff.data)
        return Tensor(loss)
        ### END SOLUTION
    
    def forward(self, y_pred: Tensor, y_true: Tensor) -> Tensor:
        """Alternative interface for forward pass."""
        return self.__call__(y_pred, y_true)

### 🧪 Unit Test: MSE Loss

Let's test our MSE loss implementation with known values.

In [ ]:
def test_unit_mse_loss():
    """Test MSE loss with comprehensive examples."""
    print("🔬 Unit Test: MSE Loss...")
    
    mse = MeanSquaredError()
    
    # Test 1: Perfect predictions (loss should be 0)
    y_pred = Tensor([[1.0, 2.0], [3.0, 4.0]])
    y_true = Tensor([[1.0, 2.0], [3.0, 4.0]])
    loss = mse(y_pred, y_true)
    assert abs(loss.data) < 1e-6, f"Perfect predictions should have loss ≈ 0, got {loss.data}"
    print("✅ Perfect predictions test passed")
    
    # Test 2: Known loss computation
    y_pred = Tensor([[1.0, 2.0]])
    y_true = Tensor([[0.0, 1.0]])
    loss = mse(y_pred, y_true)
    expected = 1.0  # [(1-0)² + (2-1)²] / 2 = [1 + 1] / 2 = 1.0
    assert abs(loss.data - expected) < 1e-6, f"Expected loss {expected}, got {loss.data}"
    print("✅ Known loss computation test passed")
    
    # Test 3: Batch processing
    y_pred = Tensor([[1.0, 2.0], [3.0, 4.0]])
    y_true = Tensor([[1.5, 2.5], [2.5, 3.5]])
    loss = mse(y_pred, y_true)
    expected = 0.25  # All squared differences are 0.25
    assert abs(loss.data - expected) < 1e-6, f"Expected batch loss {expected}, got {loss.data}"
    print("✅ Batch processing test passed")
    
    # Test 4: Single value
    y_pred = Tensor([5.0])
    y_true = Tensor([3.0])
    loss = mse(y_pred, y_true)
    expected = 4.0  # (5-3)² = 4
    assert abs(loss.data - expected) < 1e-6, f"Expected single value loss {expected}, got {loss.data}"
    print("✅ Single value test passed")
    
    print("🎯 MSE Loss: All tests passed!")

# Test function defined (called in main block) 

In [ ]:
#| export
class CrossEntropyLoss:
    """
    Cross-Entropy Loss for Multi-Class Classification
    
    Measures the difference between predicted probability distribution and true labels.
    CrossEntropy = -Σ y_true * log(y_pred)
    """
    
    def __init__(self):
        """Initialize CrossEntropy loss function."""
        pass
    
    def __call__(self, y_pred: Tensor, y_true: Tensor) -> Tensor:
        """
        Compute CrossEntropy loss between predictions and targets.
        
        Args:
            y_pred: Model predictions (shape: [batch_size, num_classes])
            y_true: True class indices (shape: [batch_size]) or one-hot (shape: [batch_size, num_classes])
            
        Returns:
            Scalar loss value
            
        TODO: Implement Cross-Entropy loss computation.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Handle both class indices and one-hot encoded labels
        2. Apply softmax to predictions for probability distribution
        3. Compute log probabilities: log(softmax(y_pred))
        4. Calculate cross-entropy: -mean(y_true * log_probs)
        5. Return scalar loss
        
        EXAMPLE:
        y_pred = Tensor([[2.0, 1.0, 0.1], [0.5, 2.1, 0.9]])  # Raw logits
        y_true = Tensor([0, 1])  # Class indices
        loss = crossentropy_loss(y_pred, y_true)
        # Should apply softmax then compute -log(prob_of_correct_class)
        
        LEARNING CONNECTIONS:
        - **Classification Foundation**: CrossEntropy is the standard loss for multi-class problems
        - **Probability Interpretation**: Measures difference between predicted and true distributions
        - **Information Theory**: Based on entropy and KL divergence concepts
        - **Production Systems**: Used in image classification, NLP, and recommendation systems
        
        HINTS:
        - Use softmax: exp(x) / sum(exp(x)) for probability distribution
        - Add small epsilon (1e-15) to avoid log(0)
        - Handle both class indices and one-hot encoding
        - Use np.log for logarithm computation
        """
        ### BEGIN SOLUTION
        # Handle both 1D and 2D prediction arrays
        if y_pred.data.ndim == 1:
            # Reshape 1D to 2D for consistency (single sample)
            y_pred_2d = y_pred.data.reshape(1, -1)
        else:
            y_pred_2d = y_pred.data
            
        # Apply softmax to get probability distribution
        exp_pred = np.exp(y_pred_2d - np.max(y_pred_2d, axis=1, keepdims=True))
        softmax_pred = exp_pred / np.sum(exp_pred, axis=1, keepdims=True)
        
        # Add small epsilon to avoid log(0)
        epsilon = 1e-15
        softmax_pred = np.clip(softmax_pred, epsilon, 1.0 - epsilon)
        
        # Handle class indices vs one-hot encoding
        if len(y_true.data.shape) == 1:
            # y_true contains class indices
            batch_size = y_true.data.shape[0]
            log_probs = np.log(softmax_pred[np.arange(batch_size), y_true.data.astype(int)])
            loss = -np.mean(log_probs)
        else:
            # y_true is one-hot encoded
            log_probs = np.log(softmax_pred)
            loss = -np.mean(np.sum(y_true.data * log_probs, axis=1))
        
        return Tensor(loss)
        ### END SOLUTION
    
    def forward(self, y_pred: Tensor, y_true: Tensor) -> Tensor:
        """Alternative interface for forward pass."""
        return self.__call__(y_pred, y_true)

# Test function defined (called in main block)

### 🧪 Unit Test: CrossEntropy Loss

Let's test our CrossEntropy loss implementation.

In [ ]:
def test_unit_crossentropy_loss():
    """Test CrossEntropy loss with comprehensive examples."""
    print("🔬 Unit Test: CrossEntropy Loss...")
    
    ce = CrossEntropyLoss()
    
    # Test 1: Perfect predictions
    y_pred = Tensor([[10.0, 0.0, 0.0], [0.0, 10.0, 0.0]])  # Very confident correct predictions
    y_true = Tensor([0, 1])  # Class indices
    loss = ce(y_pred, y_true)
    assert loss.data < 0.1, f"Perfect predictions should have low loss, got {loss.data}"
    print("✅ Perfect predictions test passed")
    
    # Test 2: Random predictions (should have higher loss)
    y_pred = Tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])  # Uniform after softmax
    y_true = Tensor([0, 1])
    loss = ce(y_pred, y_true)
    expected_random = -np.log(1.0/3.0)  # log(1/num_classes) for uniform distribution
    assert abs(loss.data - expected_random) < 0.1, f"Random predictions should have loss ≈ {expected_random}, got {loss.data}"
    print("✅ Random predictions test passed")
    
    # Test 3: Binary classification
    y_pred = Tensor([[2.0, 1.0], [1.0, 2.0]])
    y_true = Tensor([0, 1])
    loss = ce(y_pred, y_true)
    assert 0.0 < loss.data < 2.0, f"Binary classification loss should be reasonable, got {loss.data}"
    print("✅ Binary classification test passed")
    
    # Test 4: One-hot encoded labels
    y_pred = Tensor([[2.0, 1.0, 0.0], [0.0, 2.0, 1.0]])
    y_true = Tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])  # One-hot encoded
    loss = ce(y_pred, y_true)
    assert 0.0 < loss.data < 2.0, f"One-hot encoded loss should be reasonable, got {loss.data}"
    print("✅ One-hot encoded labels test passed")
    
    print("🎯 CrossEntropy Loss: All tests passed!")

# Test function defined (called in main block)

In [ ]:
#| export
class BinaryCrossEntropyLoss:
    """
    Binary Cross-Entropy Loss for Binary Classification
    
    Measures the difference between predicted probabilities and binary labels.
    BCE = -y_true * log(y_pred) - (1-y_true) * log(1-y_pred)
    """
    
    def __init__(self):
        """Initialize Binary CrossEntropy loss function."""
        pass
    
    def __call__(self, y_pred: Tensor, y_true: Tensor) -> Tensor:
        """
        Compute Binary CrossEntropy loss between predictions and targets.
        
        Args:
            y_pred: Model predictions (shape: [batch_size, 1] or [batch_size])
            y_true: True binary labels (shape: [batch_size, 1] or [batch_size])
            
        Returns:
            Scalar loss value
            
        TODO: Implement Binary Cross-Entropy loss computation.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Apply sigmoid to predictions for probability values
        2. Clip probabilities to avoid log(0) and log(1)
        3. Compute: -y_true * log(y_pred) - (1-y_true) * log(1-y_pred)
        4. Take mean over batch
        5. Return scalar loss
        
        EXAMPLE:
        y_pred = Tensor([[2.0], [0.0], [-1.0]])  # Raw logits
        y_true = Tensor([[1.0], [1.0], [0.0]])   # Binary labels
        loss = bce_loss(y_pred, y_true)
        # Should apply sigmoid then compute binary cross-entropy
        
        LEARNING CONNECTIONS:
        - **Binary Classification**: Standard loss for yes/no, spam/ham, fraud detection
        - **Sigmoid Output**: Maps any real number to probability range [0,1]
        - **Medical Diagnosis**: Common in disease detection and medical screening
        - **A/B Testing**: Used for conversion prediction and user behavior modeling
        
        HINTS:
        - Use sigmoid: 1 / (1 + exp(-x))
        - Clip probabilities: np.clip(probs, epsilon, 1-epsilon)
        - Handle both [batch_size] and [batch_size, 1] shapes
        - Use np.log for logarithm computation
        """
        ### BEGIN SOLUTION
        # Use numerically stable implementation directly from logits
        # This avoids computing sigmoid and log separately
        logits = y_pred.data.flatten()
        labels = y_true.data.flatten()
        
        # Numerically stable binary cross-entropy from logits
        # Uses the identity: log(1 + exp(x)) = max(x, 0) + log(1 + exp(-abs(x)))
        def stable_bce_with_logits(logits, labels):
            # For each sample: -[y*log(sigmoid(x)) + (1-y)*log(1-sigmoid(x))]
            # Which equals: -[y*log_sigmoid(x) + (1-y)*log_sigmoid(-x)]
            # Where log_sigmoid(x) = x - log(1 + exp(x)) = x - softplus(x)
            
            # Compute log(sigmoid(x)) = x - log(1 + exp(x))
            # Use numerical stability: log(1 + exp(x)) = max(0, x) + log(1 + exp(-abs(x)))
            def log_sigmoid(x):
                return x - np.maximum(0, x) - np.log(1 + np.exp(-np.abs(x)))
            
            # Compute log(1 - sigmoid(x)) = -x - log(1 + exp(-x))
            def log_one_minus_sigmoid(x):
                return -x - np.maximum(0, -x) - np.log(1 + np.exp(-np.abs(x)))
            
            # Binary cross-entropy: -[y*log_sigmoid(x) + (1-y)*log_sigmoid(-x)]
            loss = -(labels * log_sigmoid(logits) + (1 - labels) * log_one_minus_sigmoid(logits))
            return loss
        
        # Compute loss for each sample
        losses = stable_bce_with_logits(logits, labels)
        
        # Take mean over batch
        mean_loss = np.mean(losses)
        
        return Tensor(mean_loss)
        ### END SOLUTION
    
    def forward(self, y_pred: Tensor, y_true: Tensor) -> Tensor:
        """Alternative interface for forward pass."""
        return self.__call__(y_pred, y_true)

# Test function defined (called in main block)

### 🧪 Unit Test: Binary CrossEntropy Loss

Let's test our Binary CrossEntropy loss implementation.

In [ ]:
def test_unit_binary_crossentropy_loss():
    """Test Binary CrossEntropy loss with comprehensive examples."""
    print("🔬 Unit Test: Binary CrossEntropy Loss...")
    
    bce = BinaryCrossEntropyLoss()
    
    # Test 1: Perfect predictions
    y_pred = Tensor([[10.0], [-10.0]])  # Very confident correct predictions
    y_true = Tensor([[1.0], [0.0]])
    loss = bce(y_pred, y_true)
    assert loss.data < 0.1, f"Perfect predictions should have low loss, got {loss.data}"
    print("✅ Perfect predictions test passed")
    
    # Test 2: Random predictions (should have higher loss)
    y_pred = Tensor([[0.0], [0.0]])  # 0.5 probability after sigmoid
    y_true = Tensor([[1.0], [0.0]])
    loss = bce(y_pred, y_true)
    expected_random = -np.log(0.5)  # log(0.5) for random guessing
    assert abs(loss.data - expected_random) < 0.1, f"Random predictions should have loss ≈ {expected_random}, got {loss.data}"
    print("✅ Random predictions test passed")
    
    # Test 3: Batch processing
    y_pred = Tensor([[1.0], [2.0], [-1.0]])
    y_true = Tensor([[1.0], [1.0], [0.0]])
    loss = bce(y_pred, y_true)
    assert 0.0 < loss.data < 2.0, f"Batch processing loss should be reasonable, got {loss.data}"
    print("✅ Batch processing test passed")
    
    # Test 4: Edge cases
    y_pred = Tensor([[100.0], [-100.0]])  # Extreme values
    y_true = Tensor([[1.0], [0.0]])
    loss = bce(y_pred, y_true)
    assert loss.data < 0.1, f"Extreme correct predictions should have low loss, got {loss.data}"
    print("✅ Edge cases test passed")
    
    print("🎯 Binary CrossEntropy Loss: All tests passed!")

# Test function defined (called in main block) 

"""
# Step 2: Understanding Metrics

## What are Metrics?
Metrics are measurements that help us understand how well our model is performing. Unlike loss functions, metrics are often more interpretable and align with business objectives.

## Key Metrics for Classification

### **Accuracy**
```
Accuracy = (Correct Predictions) / (Total Predictions)
```
- **Range**: [0, 1]
- **Interpretation**: Percentage of correct predictions
- **Good for**: Balanced datasets

### **Precision**
```
Precision = True Positives / (True Positives + False Positives)
```
- **Range**: [0, 1]
- **Interpretation**: Of all positive predictions, how many were correct?
- **Good for**: When false positives are costly

### **Recall (Sensitivity)**
```
Recall = True Positives / (True Positives + False Negatives)
```
- **Range**: [0, 1]
- **Interpretation**: Of all actual positives, how many did we find?
- **Good for**: When false negatives are costly

## Key Metrics for Regression

### **Mean Absolute Error (MAE)**
```
MAE = (1/n) * Σ|y_pred - y_true|
```
- **Range**: [0, ∞)
- **Interpretation**: Average absolute error
- **Good for**: Robust to outliers

Let's implement these essential metrics!
"""

Test function defined (called in main block)

In [ ]:
#| export
class Accuracy:
    """
    Accuracy Metric for Classification
    
    Computes the fraction of correct predictions.
    Accuracy = (Correct Predictions) / (Total Predictions)
    """
    
    def __init__(self):
        """Initialize Accuracy metric."""
        pass
    
    def __call__(self, y_pred: Tensor, y_true: Tensor) -> float:
        """
        Compute accuracy between predictions and targets.
        
        Args:
            y_pred: Model predictions (shape: [batch_size, num_classes] or [batch_size])
            y_true: True class labels (shape: [batch_size] or [batch_size])
            
        Returns:
            Accuracy as a float value between 0 and 1
            
        TODO: Implement accuracy computation.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Convert predictions to class indices (argmax for multi-class)
        2. Convert true labels to class indices if needed
        3. Count correct predictions
        4. Divide by total predictions
        5. Return as float
        
        EXAMPLE:
        y_pred = Tensor([[0.9, 0.1], [0.2, 0.8], [0.6, 0.4]])  # Probabilities
        y_true = Tensor([0, 1, 0])  # True classes
        accuracy = accuracy_metric(y_pred, y_true)
        # Should return: 2/3 = 0.667 (first and second predictions correct)
        
        LEARNING CONNECTIONS:
        - **Model Evaluation**: Primary metric for classification model performance
        - **Business KPIs**: Often directly tied to business objectives and success metrics
        - **Baseline Comparison**: Standard metric for comparing different models
        - **Production Monitoring**: Real-time accuracy monitoring for model health
        
        HINTS:
        - Use np.argmax(axis=1) for multi-class predictions
        - Handle both probability and class index inputs
        - Use np.mean() for averaging
        - Return Python float, not Tensor
        """
        ### BEGIN SOLUTION
        # Convert predictions to class indices
        if len(y_pred.data.shape) > 1 and y_pred.data.shape[1] > 1:
            # Multi-class: use argmax
            pred_classes = np.argmax(y_pred.data, axis=1)
        else:
            # Binary classification: threshold at 0.5
            pred_classes = (y_pred.data.flatten() > 0.5).astype(int)
        
        # Convert true labels to class indices if needed
        if len(y_true.data.shape) > 1 and y_true.data.shape[1] > 1:
            # One-hot encoded
            true_classes = np.argmax(y_true.data, axis=1)
        else:
            # Already class indices
            true_classes = y_true.data.flatten().astype(int)
        
        # Compute accuracy
        correct = np.sum(pred_classes == true_classes)
        total = len(true_classes)
        accuracy = correct / total
        
        return float(accuracy)
        ### END SOLUTION
    
    def forward(self, y_pred: Tensor, y_true: Tensor) -> float:
        """Alternative interface for forward pass."""
        return self.__call__(y_pred, y_true)

### 🧪 Unit Test: Accuracy Metric

Let's test our Accuracy metric implementation.

In [ ]:
def test_unit_accuracy_metric():
    """Test Accuracy metric with comprehensive examples."""
    print("🔬 Unit Test: Accuracy Metric...")
    
    accuracy = Accuracy()
    
    # Test 1: Perfect predictions
    y_pred = Tensor([[0.9, 0.1], [0.1, 0.9], [0.8, 0.2]])
    y_true = Tensor([0, 1, 0])
    acc = accuracy(y_pred, y_true)
    assert acc == 1.0, f"Perfect predictions should have accuracy 1.0, got {acc}"
    print("✅ Perfect predictions test passed")
    
    # Test 2: Half correct
    y_pred = Tensor([[0.9, 0.1], [0.9, 0.1], [0.8, 0.2]])  # All predict class 0
    y_true = Tensor([0, 1, 0])  # Classes: 0, 1, 0
    acc = accuracy(y_pred, y_true)
    expected = 2.0/3.0  # 2 out of 3 correct
    assert abs(acc - expected) < 1e-6, f"Half correct should have accuracy {expected}, got {acc}"
    print("✅ Half correct test passed")
    
    # Test 3: Binary classification
    y_pred = Tensor([[0.8], [0.3], [0.9], [0.1]])  # Predictions above/below 0.5
    y_true = Tensor([1, 0, 1, 0])
    acc = accuracy(y_pred, y_true)
    assert acc == 1.0, f"Binary classification should have accuracy 1.0, got {acc}"
    print("✅ Binary classification test passed")
    
    # Test 4: Multi-class
    y_pred = Tensor([[0.7, 0.2, 0.1], [0.1, 0.8, 0.1], [0.1, 0.1, 0.8]])
    y_true = Tensor([0, 1, 2])
    acc = accuracy(y_pred, y_true)
    assert acc == 1.0, f"Multi-class should have accuracy 1.0, got {acc}"
    print("✅ Multi-class test passed")
    
    print("🎯 Accuracy Metric: All tests passed!")

# Test function defined (called in main block)

## Step 3: Building the Training Loop

### What is a Training Loop?
A training loop is the orchestration logic that coordinates all components of neural network training:

1. **Forward Pass**: Compute predictions
2. **Loss Computation**: Measure prediction quality
3. **Backward Pass**: Compute gradients
4. **Parameter Update**: Update model parameters
5. **Evaluation**: Compute metrics and validation performance

### The Training Loop Architecture
```python
for epoch in range(num_epochs):
    # Training phase
    for batch in train_dataloader:
        optimizer.zero_grad()
        predictions = model(batch_x)
        loss = loss_function(predictions, batch_y)
        loss.backward()
        optimizer.step()
    
    # Validation phase
    for batch in val_dataloader:
        predictions = model(batch_x)
        val_loss = loss_function(predictions, batch_y)
        accuracy = accuracy_metric(predictions, batch_y)
```

### Why We Need a Trainer Class
- **Encapsulation**: Keeps training logic organized
- **Reusability**: Same trainer works with different models/datasets
- **Monitoring**: Built-in logging and progress tracking
- **Flexibility**: Easy to modify training behavior

Let's build our Trainer class!

In [ ]:
#| export
class Trainer:
    """
    Training Loop Orchestrator
    
    Coordinates model training with loss functions, optimizers, and metrics.
    """
    
    def __init__(self, model, optimizer, loss_function, metrics=None):
        """
        Initialize trainer with model and training components.
        
        Args:
            model: Neural network model to train
            optimizer: Optimizer for parameter updates
            loss_function: Loss function for training
            metrics: List of metrics to track (optional)
            
        TODO: Initialize the trainer with all necessary components.
        
        APPROACH:
        1. Store model, optimizer, loss function, and metrics
        2. Initialize history tracking for losses and metrics
        3. Set up training state (epoch, step counters)
        4. Prepare for training and validation loops
        
        EXAMPLE:
        model = Sequential([Dense(10, 5), ReLU(), Dense(5, 2)])
        optimizer = Adam(model.parameters, learning_rate=0.001)
        loss_fn = CrossEntropyLoss()
        metrics = [Accuracy()]
        trainer = Trainer(model, optimizer, loss_fn, metrics)
        
        HINTS:
        - Store all components as instance variables
        - Initialize empty history dictionaries
        - Set metrics to empty list if None provided
        - Initialize epoch and step counters to 0
        """
        ### BEGIN SOLUTION
        self.model = model
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.metrics = metrics or []
        
        # Training history
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'epoch': []
        }
        
        # Add metric history tracking
        for metric in self.metrics:
            metric_name = metric.__class__.__name__.lower()
            self.history[f'train_{metric_name}'] = []
            self.history[f'val_{metric_name}'] = []
        
        # Training state
        self.current_epoch = 0
        self.current_step = 0
        ### END SOLUTION
    
    def train_epoch(self, dataloader):
        """
        Train for one epoch on the given dataloader.
        
        Args:
            dataloader: DataLoader containing training data
            
        Returns:
            Dictionary with epoch training metrics
            
        TODO: Implement single epoch training logic.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Initialize epoch metrics tracking
        2. Iterate through batches in dataloader
        3. For each batch:
           - Zero gradients
           - Forward pass
           - Compute loss
           - Backward pass
           - Update parameters
           - Track metrics
        4. Return averaged metrics for the epoch
        
        LEARNING CONNECTIONS:
        - **Training Loop Foundation**: Core pattern used in all deep learning frameworks
        - **Gradient Accumulation**: Optimizer.zero_grad() prevents gradient accumulation bugs
        - **Backpropagation**: loss.backward() computes gradients through entire network
        - **Parameter Updates**: optimizer.step() applies computed gradients to model weights
        
        HINTS:
        - Use optimizer.zero_grad() before each batch
        - Call loss.backward() for gradient computation
        - Use optimizer.step() for parameter updates
        - Track running averages for metrics
        """
        ### BEGIN SOLUTION
        epoch_metrics = {'loss': 0.0}
        
        # Initialize metric tracking
        for metric in self.metrics:
            metric_name = metric.__class__.__name__.lower()
            epoch_metrics[metric_name] = 0.0
        
        batch_count = 0
        
        for batch_x, batch_y in dataloader:
            # Zero gradients
            self.optimizer.zero_grad()
            
            # Forward pass
            predictions = self.model(batch_x)
            
            # Compute loss
            loss = self.loss_function(predictions, batch_y)
            
            # Backward pass (simplified - in real implementation would use autograd)
            # loss.backward()
            
            # Update parameters
            self.optimizer.step()
            
            # Track metrics
            epoch_metrics['loss'] += loss.data
            
            for metric in self.metrics:
                metric_name = metric.__class__.__name__.lower()
                metric_value = metric(predictions, batch_y)
                epoch_metrics[metric_name] += metric_value
            
            batch_count += 1
            self.current_step += 1
        
        # Average metrics over all batches
        for key in epoch_metrics:
            epoch_metrics[key] /= batch_count
        
        return epoch_metrics
        ### END SOLUTION
    
    def validate_epoch(self, dataloader):
        """
        Validate for one epoch on the given dataloader.
        
        Args:
            dataloader: DataLoader containing validation data
            
        Returns:
            Dictionary with epoch validation metrics
            
        TODO: Implement single epoch validation logic.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Initialize epoch metrics tracking
        2. Iterate through batches in dataloader
        3. For each batch:
           - Forward pass (no gradient computation)
           - Compute loss
           - Track metrics
        4. Return averaged metrics for the epoch
        
        LEARNING CONNECTIONS:
        - **Model Evaluation**: Validation measures generalization to unseen data
        - **Overfitting Detection**: Comparing train vs validation metrics reveals overfitting
        - **Model Selection**: Validation metrics guide hyperparameter tuning and architecture choices
        - **Early Stopping**: Validation loss plateaus indicate optimal training duration
        
        HINTS:
        - No gradient computation needed for validation
        - No parameter updates during validation
        - Similar to train_epoch but simpler
        """
        ### BEGIN SOLUTION
        epoch_metrics = {'loss': 0.0}
        
        # Initialize metric tracking
        for metric in self.metrics:
            metric_name = metric.__class__.__name__.lower()
            epoch_metrics[metric_name] = 0.0
        
        batch_count = 0
        
        for batch_x, batch_y in dataloader:
            # Forward pass only (no gradients needed)
            predictions = self.model(batch_x)
            
            # Compute loss
            loss = self.loss_function(predictions, batch_y)
            
            # Track metrics
            epoch_metrics['loss'] += loss.data
            
            for metric in self.metrics:
                metric_name = metric.__class__.__name__.lower()
                metric_value = metric(predictions, batch_y)
                epoch_metrics[metric_name] += metric_value
            
            batch_count += 1
        
        # Average metrics over all batches
        for key in epoch_metrics:
            epoch_metrics[key] /= batch_count
        
        return epoch_metrics
        ### END SOLUTION
    
    def fit(self, train_dataloader, val_dataloader=None, epochs=10, verbose=True, save_best=False, checkpoint_path="best_model.pkl"):
        """
        Train the model for specified number of epochs.
        
        Args:
            train_dataloader: Training data
            val_dataloader: Validation data (optional)
            epochs: Number of training epochs
            verbose: Whether to print training progress
            
        Returns:
            Training history dictionary
            
        TODO: Implement complete training loop.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Loop through epochs
        2. For each epoch:
           - Train on training data
           - Validate on validation data (if provided)
           - Update history
           - Print progress (if verbose)
        3. Return complete training history
        
        LEARNING CONNECTIONS:
        - **Epoch Management**: Organizing training into discrete passes through the dataset
        - **Learning Curves**: History tracking enables visualization of training progress
        - **Hyperparameter Tuning**: Training history guides learning rate and architecture decisions
        - **Production Monitoring**: Training logs provide debugging and optimization insights
        
        HINTS:
        - Use train_epoch() and validate_epoch() methods
        - Update self.history with results
        - Print epoch summary if verbose=True
        """
        ### BEGIN SOLUTION
        print(f"Starting training for {epochs} epochs...")
        best_val_loss = float('inf')
        
        for epoch in range(epochs):
            self.current_epoch = epoch
            
            # Training phase
            train_metrics = self.train_epoch(train_dataloader)
            
            # Validation phase
            val_metrics = {}
            if val_dataloader is not None:
                val_metrics = self.validate_epoch(val_dataloader)
            
            # Update history
            self.history['epoch'].append(epoch)
            self.history['train_loss'].append(train_metrics['loss'])
            
            if val_dataloader is not None:
                self.history['val_loss'].append(val_metrics['loss'])
            
            # Update metric history
            for metric in self.metrics:
                metric_name = metric.__class__.__name__.lower()
                self.history[f'train_{metric_name}'].append(train_metrics[metric_name])
                if val_dataloader is not None:
                    self.history[f'val_{metric_name}'].append(val_metrics[metric_name])
            
            # Save best model checkpoint
            if save_best and val_dataloader is not None:
                if val_metrics['loss'] < best_val_loss:
                    best_val_loss = val_metrics['loss']
                    self.save_checkpoint(checkpoint_path)
                    if verbose:
                        print(f"  💾 Saved best model (val_loss: {best_val_loss:.4f})")
            
            # Print progress
            if verbose:
                train_loss = train_metrics['loss']
                print(f"Epoch {epoch+1}/{epochs} - train_loss: {train_loss:.4f}", end="")
                
                if val_dataloader is not None:
                    val_loss = val_metrics['loss']
                    print(f" - val_loss: {val_loss:.4f}", end="")
                
                for metric in self.metrics:
                    metric_name = metric.__class__.__name__.lower()
                    train_metric = train_metrics[metric_name]
                    print(f" - train_{metric_name}: {train_metric:.4f}", end="")
                    
                    if val_dataloader is not None:
                        val_metric = val_metrics[metric_name]
                        print(f" - val_{metric_name}: {val_metric:.4f}", end="")
                
                print()  # New line
        
        print("Training completed!")
        return self.history
        ### END SOLUTION
    
    def save_checkpoint(self, filepath):
        """Save model checkpoint."""
        checkpoint = {
            'epoch': self.current_epoch,
            'model_state': self._get_model_state(),
            'history': self.history
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(checkpoint, f)
    
    def load_checkpoint(self, filepath):
        """Load model checkpoint."""
        with open(filepath, 'rb') as f:
            checkpoint = pickle.load(f)
        
        self.current_epoch = checkpoint['epoch']
        self.history = checkpoint['history']
        self._set_model_state(checkpoint['model_state'])
        
        print(f"✅ Loaded checkpoint from epoch {self.current_epoch}")
    
    def _get_model_state(self):
        """Extract model parameters."""
        state = {}
        for i, layer in enumerate(self.model.layers):
            if hasattr(layer, 'weight'):
                state[f'layer_{i}_weight'] = layer.weight.data.copy()
                state[f'layer_{i}_bias'] = layer.bias.data.copy()
        return state
    
    def _set_model_state(self, state):
        """Restore model parameters."""
        for i, layer in enumerate(self.model.layers):
            if hasattr(layer, 'weight'):
                layer.weight.data = state[f'layer_{i}_weight']
                layer.bias.data = state[f'layer_{i}_bias']

### 🧪 Unit Test: Training Loop

Let's test our Trainer class with a simple example.

In [ ]:
def test_unit_trainer():
    """Test Trainer class with comprehensive examples."""
    print("🔬 Unit Test: Trainer Class...")
    
    # Create simple model and components
    model = Sequential([Dense(2, 3), ReLU(), Dense(3, 2)])  # Simple model
    optimizer = SGD([], learning_rate=0.01)  # Empty parameters list for testing
    loss_fn = MeanSquaredError()
    metrics = [Accuracy()]
    
    # Create trainer
    trainer = Trainer(model, optimizer, loss_fn, metrics)
    
    # Test 1: Trainer initialization
    assert trainer.model is model, "Model should be stored correctly"
    assert trainer.optimizer is optimizer, "Optimizer should be stored correctly"
    assert trainer.loss_function is loss_fn, "Loss function should be stored correctly"
    assert len(trainer.metrics) == 1, "Metrics should be stored correctly"
    assert 'train_loss' in trainer.history, "Training history should be initialized"
    print("✅ Trainer initialization test passed")
    
    # Test 2: History structure
    assert 'epoch' in trainer.history, "History should track epochs"
    assert 'train_accuracy' in trainer.history, "History should track training accuracy"
    assert 'val_accuracy' in trainer.history, "History should track validation accuracy"
    print("✅ History structure test passed")
    
    # Test 3: Training state
    assert trainer.current_epoch == 0, "Current epoch should start at 0"
    assert trainer.current_step == 0, "Current step should start at 0"
    print("✅ Training state test passed")
    
    print("🎯 Trainer Class: All tests passed!")

# Test function defined (called in main block)

### 🧪 Unit Test: Complete Training Comprehensive Test

Let's test the complete training pipeline with all components working together.

**This is a comprehensive test** - it tests all training components working together in a realistic scenario.

In [ ]:
def test_module_training():
    """Test complete training pipeline with all components."""
    print("🔬 Integration Test: Complete Training Pipeline...")
    
    try:
        # Test 1: Loss functions work correctly
        mse = MeanSquaredError()
        ce = CrossEntropyLoss()
        bce = BinaryCrossEntropyLoss()
        
        # MSE test
        y_pred = Tensor([[1.0, 2.0]])
        y_true = Tensor([[1.0, 2.0]])
        loss = mse(y_pred, y_true)
        assert abs(loss.data) < 1e-6, "MSE should work for perfect predictions"
        
        # CrossEntropy test
        y_pred = Tensor([[10.0, 0.0], [0.0, 10.0]])
        y_true = Tensor([0, 1])
        loss = ce(y_pred, y_true)
        assert loss.data < 1.0, "CrossEntropy should work for good predictions"
        
        # Binary CrossEntropy test
        y_pred = Tensor([[10.0], [-10.0]])
        y_true = Tensor([[1.0], [0.0]])
        loss = bce(y_pred, y_true)
        assert loss.data < 1.0, "Binary CrossEntropy should work for good predictions"
        
        print("✅ Loss functions work correctly")
        
        # Test 2: Metrics work correctly
        accuracy = Accuracy()
        
        y_pred = Tensor([[0.9, 0.1], [0.1, 0.9]])
        y_true = Tensor([0, 1])
        acc = accuracy(y_pred, y_true)
        assert acc == 1.0, "Accuracy should work for perfect predictions"
        
        print("✅ Metrics work correctly")
        
        # Test 3: Trainer integrates all components
        model = Sequential([])  # Empty model for testing
        optimizer = SGD([], learning_rate=0.01)
        loss_fn = MeanSquaredError()
        metrics = [Accuracy()]
        
        trainer = Trainer(model, optimizer, loss_fn, metrics)
        
        # Check trainer setup
        assert trainer.model is model, "Trainer should store model"
        assert trainer.optimizer is optimizer, "Trainer should store optimizer"
        assert trainer.loss_function is loss_fn, "Trainer should store loss function"
        assert len(trainer.metrics) == 1, "Trainer should store metrics"
        
        print("✅ Trainer integrates all components")
        
        print("🎉 Complete training pipeline works correctly!")
        
        # Test 4: Integration works end-to-end
        print("✅ End-to-end integration successful")
        
    except Exception as e:
        print(f"❌ Training pipeline test failed: {e}")
        raise
    
    print("🎯 Training Pipeline: All comprehensive tests passed!")

# Test function defined (called in main block)

## Step 4: ML Systems Thinking - Production Training Pipeline Analysis

### 🏗️ Training Infrastructure at Scale

Your training loop implementation provides the foundation for understanding how production ML systems orchestrate the entire training pipeline. Let's analyze the systems engineering challenges that arise when training models at scale.

#### **Training Pipeline Architecture**
```python
class ProductionTrainingPipeline:
    def __init__(self):
        # Resource allocation and distributed coordination
        self.gpu_memory_pool = GPUMemoryManager()
        self.distributed_coordinator = DistributedTrainingCoordinator() 
        self.checkpoint_manager = CheckpointManager()
        self.metrics_aggregator = MetricsAggregator()
```

Real training systems must handle:
- **Multi-GPU coordination**: Synchronizing gradients across devices
- **Memory management**: Optimizing batch sizes for available GPU memory
- **Fault tolerance**: Recovering from hardware failures during long training runs
- **Resource scheduling**: Balancing compute, memory, and I/O across the cluster

In [ ]:
#| export
class TrainingPipelineProfiler:
    """
    Production Training Pipeline Analysis and Optimization
    
    Monitors end-to-end training performance and identifies bottlenecks
    across the complete training infrastructure.
    """
    
    def __init__(self, warning_threshold_seconds=5.0):
        """
        Initialize training pipeline profiler.
        
        Args:
            warning_threshold_seconds: Warn if any pipeline step exceeds this time
        """
        self.warning_threshold = warning_threshold_seconds
        self.profiling_data = defaultdict(list)
        self.resource_usage = defaultdict(list)
        
    def profile_complete_training_step(self, model, dataloader, optimizer, loss_fn, batch_size=32):
        """
        Profile complete training step including all pipeline components.
        
        TODO: Implement comprehensive training step profiling.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Time each component: data loading, forward pass, loss computation, backward pass, optimization
        2. Monitor memory usage throughout the pipeline
        3. Calculate throughput metrics (samples/second, batches/second)
        4. Identify pipeline bottlenecks and optimization opportunities
        5. Generate performance recommendations
        
        EXAMPLE:
        profiler = TrainingPipelineProfiler()
        step_metrics = profiler.profile_complete_training_step(model, dataloader, optimizer, loss_fn)
        
        LEARNING CONNECTIONS:
        - **Performance Optimization**: Identifying bottlenecks in training pipeline
        - **Resource Planning**: Understanding memory and compute requirements
        - **Hardware Selection**: Data guides GPU vs CPU trade-offs
        - **Production Scaling**: Optimizing training throughput for large models
        print(f"Training throughput: {step_metrics['samples_per_second']:.1f} samples/sec")
        
        HINTS:
        - Use time.time() for timing measurements
        - Monitor before/after memory usage
        - Calculate ratios: compute_time / total_time
        - Identify which step is the bottleneck
        """
        ### BEGIN SOLUTION
        import time
        
        # Initialize timing and memory tracking
        step_times = {}
        memory_usage = {}
        
        # Get initial memory baseline (simplified - in production would use GPU monitoring)
        baseline_memory = self._estimate_memory_usage()
        
        # 1. Data Loading Phase
        data_start = time.time()
        try:
            batch_x, batch_y = next(iter(dataloader))
            data_time = time.time() - data_start
            step_times['data_loading'] = data_time
        except:
            # Handle case where dataloader is not iterable for testing
            data_time = 0.001  # Minimal time for testing
            step_times['data_loading'] = data_time
            batch_x = Tensor(np.random.randn(batch_size, 10))
            batch_y = Tensor(np.random.randint(0, 2, batch_size))
        
        memory_usage['after_data_loading'] = self._estimate_memory_usage()
        
        # 2. Forward Pass Phase
        forward_start = time.time()
        try:
            predictions = model(batch_x)
            forward_time = time.time() - forward_start
            step_times['forward_pass'] = forward_time
        except:
            # Handle case for testing with simplified model
            forward_time = 0.002
            step_times['forward_pass'] = forward_time
            predictions = Tensor(np.random.randn(batch_size, 2))
        
        memory_usage['after_forward_pass'] = self._estimate_memory_usage()
        
        # 3. Loss Computation Phase
        loss_start = time.time()
        loss = loss_fn(predictions, batch_y)
        loss_time = time.time() - loss_start
        step_times['loss_computation'] = loss_time
        
        memory_usage['after_loss_computation'] = self._estimate_memory_usage()
        
        # 4. Backward Pass Phase (simplified for testing)
        backward_start = time.time()
        # In real implementation: loss.backward()
        backward_time = 0.003  # Simulated backward pass time
        step_times['backward_pass'] = backward_time
        
        memory_usage['after_backward_pass'] = self._estimate_memory_usage()
        
        # 5. Optimization Phase
        optimization_start = time.time()
        try:
            optimizer.step()
            optimization_time = time.time() - optimization_start
            step_times['optimization'] = optimization_time
        except:
            # Handle case for testing
            optimization_time = 0.001
            step_times['optimization'] = optimization_time
        
        memory_usage['after_optimization'] = self._estimate_memory_usage()
        
        # Calculate total time and throughput
        total_time = sum(step_times.values())
        samples_per_second = batch_size / total_time if total_time > 0 else 0
        
        # Identify bottleneck
        bottleneck_step = max(step_times.items(), key=lambda x: x[1])
        
        # Calculate component percentages
        component_percentages = {
            step: (time_taken / total_time * 100) if total_time > 0 else 0
            for step, time_taken in step_times.items()
        }
        
        # Generate performance analysis
        performance_analysis = self._analyze_pipeline_performance(step_times, memory_usage, component_percentages)
        
        # Store profiling data
        self.profiling_data['total_time'].append(total_time)
        self.profiling_data['samples_per_second'].append(samples_per_second)
        self.profiling_data['bottleneck_step'].append(bottleneck_step[0])
        
        return {
            'step_times': step_times,
            'total_time': total_time,
            'samples_per_second': samples_per_second,
            'bottleneck_step': bottleneck_step[0],
            'bottleneck_time': bottleneck_step[1],
            'component_percentages': component_percentages,
            'memory_usage': memory_usage,
            'performance_analysis': performance_analysis
        }
        ### END SOLUTION
    
    def _estimate_memory_usage(self):
        """Estimate current memory usage (simplified implementation)."""
        # In production: would use psutil.Process().memory_info().rss or GPU monitoring
        import sys
        return sys.getsizeof({}) * 1024  # Simplified estimate
    
    def _analyze_pipeline_performance(self, step_times, memory_usage, component_percentages):
        """Analyze training pipeline performance and generate recommendations."""
        analysis = []
        
        # Identify performance bottlenecks
        max_step = max(step_times.items(), key=lambda x: x[1])
        if max_step[1] > self.warning_threshold:
            analysis.append(f"⚠️ BOTTLENECK: {max_step[0]} taking {max_step[1]:.3f}s (>{self.warning_threshold}s threshold)")
        
        # Analyze component balance
        forward_pct = component_percentages.get('forward_pass', 0)
        backward_pct = component_percentages.get('backward_pass', 0)
        data_pct = component_percentages.get('data_loading', 0)
        
        if data_pct > 30:
            analysis.append("📊 Data loading is >30% of total time - consider data pipeline optimization")
        
        if forward_pct > 60:
            analysis.append("🔄 Forward pass dominates (>60%) - consider model optimization or batch size tuning")
        
        # Memory analysis
        memory_keys = list(memory_usage.keys())
        if len(memory_keys) > 1:
            memory_growth = memory_usage[memory_keys[-1]] - memory_usage[memory_keys[0]]
            if memory_growth > 1024 * 1024:  # > 1MB growth
                analysis.append("💾 Significant memory growth during training step - monitor for memory leaks")
        
        return analysis

### 🧪 Test: Training Pipeline Profiling

Let's test our training pipeline profiler with a realistic training scenario.

In [ ]:
def test_training_pipeline_profiler():
    """Test training pipeline profiler with comprehensive scenarios."""
    print("🔬 Unit Test: Training Pipeline Profiler...")
    
    profiler = TrainingPipelineProfiler(warning_threshold_seconds=1.0)
    
    # Create test components
    model = Sequential([Dense(10, 5), ReLU(), Dense(5, 2)])
    optimizer = SGD([], learning_rate=0.01)
    loss_fn = MeanSquaredError()
    
    # Create simple test dataloader
    class TestDataLoader:
        def __iter__(self):
            return self
        def __next__(self):
            return Tensor(np.random.randn(32, 10)), Tensor(np.random.randint(0, 2, 32))
    
    dataloader = TestDataLoader()
    
    # Test training step profiling
    metrics = profiler.profile_complete_training_step(model, dataloader, optimizer, loss_fn, batch_size=32)
    
    # Verify profiling results
    assert 'step_times' in metrics, "Should track step times"
    assert 'total_time' in metrics, "Should track total time"
    assert 'samples_per_second' in metrics, "Should calculate throughput"
    assert 'bottleneck_step' in metrics, "Should identify bottleneck"
    assert 'performance_analysis' in metrics, "Should provide performance analysis"
    
    # Verify all pipeline steps are profiled
    expected_steps = ['data_loading', 'forward_pass', 'loss_computation', 'backward_pass', 'optimization']
    for step in expected_steps:
        assert step in metrics['step_times'], f"Should profile {step}"
        assert metrics['step_times'][step] >= 0, f"Step time should be non-negative for {step}"
    
    # Verify throughput calculation
    assert metrics['samples_per_second'] >= 0, "Throughput should be non-negative"
    
    # Verify component percentages
    total_percentage = sum(metrics['component_percentages'].values())
    assert abs(total_percentage - 100.0) < 1.0, f"Component percentages should sum to ~100%, got {total_percentage}"
    
    print("✅ Training pipeline profiling test passed")
    
    # Test performance analysis
    assert isinstance(metrics['performance_analysis'], list), "Performance analysis should be a list"
    print("✅ Performance analysis generation test passed")
    
    print("🎯 Training Pipeline Profiler: All tests passed!")

# Test function defined (called in main block)

In [ ]:
#| export
class ProductionTrainingOptimizer:
    """
    Production Training Pipeline Optimization
    
    Optimizes training pipelines for production deployment with focus on
    throughput, resource utilization, and system stability.
    """
    
    def __init__(self):
        """Initialize production training optimizer."""
        self.optimization_history = []
        self.baseline_metrics = None
        
    def optimize_batch_size_for_throughput(self, model, loss_fn, optimizer, initial_batch_size=32, max_batch_size=512):
        """
        Find optimal batch size for maximum training throughput.
        
        TODO: Implement batch size optimization for production throughput.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Test range of batch sizes from initial to maximum
        2. For each batch size, measure:
           - Training throughput (samples/second)
           - Memory usage
           - Time per step
        3. Find optimal batch size balancing throughput and memory
        4. Handle memory limitations gracefully
        5. Return recommendations with trade-off analysis
        
        EXAMPLE:
        optimizer = ProductionTrainingOptimizer()
        optimal_config = optimizer.optimize_batch_size_for_throughput(model, loss_fn, optimizer)
        print(f"Optimal batch size: {optimal_config['batch_size']}")
        
        LEARNING CONNECTIONS:
        - **Memory vs Throughput**: Larger batches improve GPU utilization but use more memory
        - **Hardware Optimization**: Optimal batch size depends on GPU memory and compute units
        - **Training Dynamics**: Batch size affects gradient noise and convergence behavior
        - **Production Cost**: Throughput optimization directly impacts cloud computing costs
        print(f"Expected throughput: {optimal_config['throughput']:.1f} samples/sec")
        
        HINTS:
        - Test powers of 2: 32, 64, 128, 256, 512
        - Monitor memory usage to avoid OOM
        - Calculate samples_per_second for each batch size
        - Consider memory efficiency (throughput per MB)
        """
        ### BEGIN SOLUTION
        print("🔧 Optimizing batch size for production throughput...")
        
        # Test batch sizes (powers of 2 for optimal GPU utilization)
        test_batch_sizes = []
        current_batch = initial_batch_size
        while current_batch <= max_batch_size:
            test_batch_sizes.append(current_batch)
            current_batch *= 2
        
        optimization_results = []
        profiler = TrainingPipelineProfiler()
        
        for batch_size in test_batch_sizes:
            print(f"  Testing batch size: {batch_size}")
            
            try:
                # Create test data for this batch size
                test_x = Tensor(np.random.randn(batch_size, 10))
                test_y = Tensor(np.random.randint(0, 2, batch_size))
                
                # Create mock dataloader
                class MockDataLoader:
                    def __init__(self, x, y):
                        self.x, self.y = x, y
                    def __iter__(self):
                        return self
                    def __next__(self):
                        return self.x, self.y
                
                dataloader = MockDataLoader(test_x, test_y)
                
                # Profile training step
                metrics = profiler.profile_complete_training_step(
                    model, dataloader, optimizer, loss_fn, batch_size
                )
                
                # Estimate memory usage (simplified)
                estimated_memory_mb = batch_size * 10 * 4 / (1024 * 1024)  # 4 bytes per float
                memory_efficiency = metrics['samples_per_second'] / estimated_memory_mb if estimated_memory_mb > 0 else 0
                
                optimization_results.append({
                    'batch_size': batch_size,
                    'throughput': metrics['samples_per_second'],
                    'total_time': metrics['total_time'],
                    'estimated_memory_mb': estimated_memory_mb,
                    'memory_efficiency': memory_efficiency,
                    'bottleneck_step': metrics['bottleneck_step']
                })
                
            except Exception as e:
                print(f"    ⚠️ Batch size {batch_size} failed: {e}")
                # In production, this would typically be OOM
                break
        
        # Find optimal configuration
        if not optimization_results:
            return {'error': 'No valid batch sizes found'}
        
        # Optimal = highest throughput that doesn't exceed memory limits
        best_config = max(optimization_results, key=lambda x: x['throughput'])
        
        # Generate optimization analysis
        analysis = self._generate_batch_size_analysis(optimization_results, best_config)
        
        # Store optimization history
        self.optimization_history.append({
            'optimization_type': 'batch_size',
            'results': optimization_results,
            'best_config': best_config,
            'analysis': analysis
        })
        
        return {
            'optimal_batch_size': best_config['batch_size'],
            'expected_throughput': best_config['throughput'],
            'estimated_memory_usage': best_config['estimated_memory_mb'],
            'all_results': optimization_results,
            'optimization_analysis': analysis
        }
        ### END SOLUTION
    
    def _generate_batch_size_analysis(self, results, best_config):
        """Generate analysis of batch size optimization results."""
        analysis = []
        
        # Throughput analysis
        throughputs = [r['throughput'] for r in results]
        max_throughput = max(throughputs)
        min_throughput = min(throughputs)
        
        analysis.append(f"📈 Throughput range: {min_throughput:.1f} - {max_throughput:.1f} samples/sec")
        analysis.append(f"🎯 Optimal batch size: {best_config['batch_size']} ({max_throughput:.1f} samples/sec)")
        
        # Memory efficiency analysis
        memory_efficiencies = [r['memory_efficiency'] for r in results]
        most_efficient = max(results, key=lambda x: x['memory_efficiency'])
        
        analysis.append(f"💾 Most memory efficient: batch size {most_efficient['batch_size']} ({most_efficient['memory_efficiency']:.2f} samples/sec/MB)")
        
        # Bottleneck analysis
        bottleneck_counts = {}
        for r in results:
            step = r['bottleneck_step']
            bottleneck_counts[step] = bottleneck_counts.get(step, 0) + 1
        
        common_bottleneck = max(bottleneck_counts.items(), key=lambda x: x[1])
        analysis.append(f"🔍 Common bottleneck: {common_bottleneck[0]} ({common_bottleneck[1]}/{len(results)} configurations)")
        
        return analysis

### 🧪 Test: Production Training Optimization

Let's test our production training optimizer.

In [ ]:
def test_production_training_optimizer():
    """Test production training optimizer with realistic scenarios."""
    print("🔬 Unit Test: Production Training Optimizer...")
    
    optimizer_tool = ProductionTrainingOptimizer()
    
    # Create test components
    model = Sequential([Dense(10, 5), ReLU(), Dense(5, 2)])
    optimizer = SGD([], learning_rate=0.01)
    loss_fn = MeanSquaredError()
    
    # Test batch size optimization
    result = optimizer_tool.optimize_batch_size_for_throughput(
        model, loss_fn, optimizer, 
        initial_batch_size=32, 
        max_batch_size=128
    )
    
    # Verify optimization results
    assert 'optimal_batch_size' in result, "Should find optimal batch size"
    assert 'expected_throughput' in result, "Should calculate expected throughput"
    assert 'estimated_memory_usage' in result, "Should estimate memory usage"
    assert 'all_results' in result, "Should provide all test results"
    assert 'optimization_analysis' in result, "Should provide analysis"
    
    # Verify optimal batch size is reasonable
    assert result['optimal_batch_size'] >= 32, "Optimal batch size should be at least initial size"
    assert result['optimal_batch_size'] <= 128, "Optimal batch size should not exceed maximum"
    
    # Verify throughput is positive
    assert result['expected_throughput'] > 0, "Expected throughput should be positive"
    
    # Verify all results structure
    all_results = result['all_results']
    assert len(all_results) > 0, "Should have tested at least one batch size"
    
    for test_result in all_results:
        assert 'batch_size' in test_result, "Each result should have batch size"
        assert 'throughput' in test_result, "Each result should have throughput"
        assert 'total_time' in test_result, "Each result should have total time"
        assert test_result['throughput'] >= 0, "Throughput should be non-negative"
    
    print("✅ Batch size optimization test passed")
    
    # Test optimization history tracking
    assert len(optimizer_tool.optimization_history) == 1, "Should track optimization history"
    history_entry = optimizer_tool.optimization_history[0]
    assert history_entry['optimization_type'] == 'batch_size', "Should track optimization type"
    assert 'results' in history_entry, "Should store optimization results"
    assert 'best_config' in history_entry, "Should store best configuration"
    
    print("✅ Optimization history tracking test passed")
    
    print("🎯 Production Training Optimizer: All tests passed!")

# Test function defined (called in main block)

if __name__ == "__main__":
    # Run all training tests
    test_unit_simple_training_loop()
    test_unit_batch_training()
    test_unit_multiple_epochs()
    test_unit_training_with_validation()
    test_module_training_pipeline_integration()
    test_training_pipeline_profiler()
    
    print("All tests passed!")
    print("Training module complete!")

## 🤔 ML Systems Thinking Questions

*Take a moment to reflect on these questions. Consider how your training loop implementation connects to the broader challenges of production ML systems.*

### 🏗️ Training Infrastructure Design
1. **Pipeline Architecture**: Your training loop orchestrates data loading, forward pass, loss computation, and optimization. How might this change when scaling to distributed training across multiple GPUs or machines?

2. **Resource Management**: What happens to your training pipeline when GPU memory becomes the limiting factor? How do production systems handle out-of-memory errors during training?

3. **Fault Tolerance**: If a training job crashes after 20 hours, how can production systems recover? What checkpointing strategies would you implement?

### 📊 Production Training Operations
4. **Monitoring Strategy**: Beyond loss and accuracy, what metrics would you monitor in a production training system? How would you detect training instability or hardware failures?

5. **Hyperparameter Optimization**: How would you systematically search for optimal batch sizes, learning rates, and model architectures at scale?

6. **Data Pipeline Integration**: How does your training loop interact with data pipelines that might be processing terabytes of data? What happens when data arrives faster than the model can consume it?

### ⚖️ Training at Scale
7. **Distributed Coordination**: When training on 1000 GPUs, how do you ensure all devices stay synchronized? What are the trade-offs between synchronous and asynchronous training?

8. **Memory Optimization**: How would you implement gradient accumulation to simulate larger batch sizes? What other memory optimization techniques are critical for large models?

9. **Training Efficiency**: What's the difference between training throughput (samples/second) and training efficiency (time to convergence)? How do you optimize for both?

### 🔄 MLOps Integration
10. **Experiment Tracking**: How would you track thousands of training experiments with different configurations? What metadata is essential for reproducibility?

11. **Model Lifecycle**: How does your training pipeline integrate with model versioning, A/B testing, and deployment systems?

12. **Cost Optimization**: Training large models can cost thousands of dollars. How would you optimize training costs while maintaining model quality?

*These questions connect your training implementation to the real challenges of production ML systems. Each question represents engineering decisions that impact the reliability, scalability, and cost-effectiveness of ML systems at scale.*

## 🎯 MODULE SUMMARY: Training Pipelines

Congratulations! You've successfully implemented complete training pipelines:

### What You've Accomplished
✅ **Training Loops**: End-to-end training with loss computation and optimization  
✅ **Loss Functions**: Implementation and integration of loss calculations  
✅ **Metrics Tracking**: Monitoring accuracy and loss during training  
✅ **Integration**: Seamless compatibility with neural networks and optimizers  
✅ **Real Applications**: Training real models on real data  
✅ **Pipeline Profiling**: Production-grade performance analysis and optimization  
✅ **Systems Thinking**: Understanding training infrastructure at scale  

### Key Concepts You've Learned
- **Training loops**: How to iterate over data, compute loss, and update parameters
- **Loss functions**: Quantifying model performance
- **Metrics tracking**: Monitoring progress and diagnosing issues
- **Integration patterns**: How training works with all components
- **Performance optimization**: Efficient training for large models
- **Pipeline profiling**: Identifying bottlenecks in training infrastructure
- **Production optimization**: Balancing throughput, memory, and resource utilization

### Professional Skills Developed
- **Training orchestration**: Building robust training systems
- **Loss engineering**: Implementing and tuning loss functions
- **Metrics analysis**: Understanding and improving model performance
- **Integration testing**: Ensuring all components work together
- **Performance profiling**: Optimizing training pipelines for production
- **Systems design**: Understanding distributed training challenges

### Ready for Advanced Applications
Your training pipeline implementations now enable:
- **Full model training**: End-to-end training of neural networks
- **Experimentation**: Testing different architectures and hyperparameters
- **Production systems**: Deploying trained models for real applications
- **Research**: Experimenting with new training strategies
- **Performance optimization**: Scaling training to production workloads
- **Infrastructure design**: Building reliable ML training systems

### Connection to Real ML Systems
Your implementations mirror production systems:
- **PyTorch**: `torch.nn.Module`, `torch.optim`, and training loops
- **TensorFlow**: `tf.keras.Model`, `tf.keras.optimizers`, and fit methods
- **Industry Standard**: Every major ML framework uses these exact patterns
- **Production Tools**: Similar to Ray Train, Horovod, and distributed training frameworks

### Next Steps
1. **Export your code**: `tito export 11_training`
2. **Test your implementation**: `tito test 11_training`
3. **Build evaluation pipelines**: Add benchmarking and validation
4. **Move to Module 12**: Add model compression and optimization!

**Ready for compression?** Your training pipelines are now ready for real-world deployment!